In [2]:
import polars as pl
import matplotlib.pyplot as plt
import pandas as pd
from sqlalchemy import text

from sqlalchemy import create_engine
from urllib.parse import quote_plus
import numpy as np

connection_string = (
    'mssql+pyodbc://'
    'Angel_chavez:{}@172.16.2.227\\DWHFARINTERDEV/BI_FARINTER?'
    'driver=ODBC+Driver+17+for+SQL+Server'.format(quote_plus('@ng3l_ch@v3z'))
)
engine = create_engine(connection_string)
query = text("""SELECT
    fc.Suc_Id,
    fc.Emp_Id,
    fc.Caja_Id,
    fc.Factura_Id,
    fc.Factura_Fecha,
    fc.Factura_FechaHora,
    fc.AnioMes_Id,
    fc.Cliente_Id,
    fc.MonederoTarj_Id,
    fc.Vendedor_Id,
    CASE 
        WHEN fc.Monedero_Id = '0' THEN 'No Identificada'
				WHEN  cl.Tipo_Cliente = 'Aseguradoras' THEN 'Aseguradoras'
        ELSE md.Tipo_Plan 
    END AS Tipo_Plan,
    md.Edad,
    suc.TipoSucursal_Id,
    suc.TipoSucursal_Nombre,
    suc.Zona_Id,
    suc.Zona_Nombre
FROM
    [BI_Kielsa_Hecho_FacturaEncabezado] AS fc
    JOIN [BI_Kielsa_Dim_Monedero] AS md 
        ON md.Monedero_Id = fc.Monedero_Id AND md.Emp_Id = fc.Emp_Id       
    JOIN [BI_Kielsa_Dim_Sucursal] AS suc 
        ON suc.Sucursal_Id = fc.Suc_Id AND suc.Emp_Id = fc.Emp_Id
	   JOIN [BI_Kielsa_Dim_Cliente] AS cl 
		ON cl.Emp_Id = fc.Emp_Id AND cl.Cliente_Id = fc.Cliente_Id
WHERE
    fc.Factura_Fecha >= '2024-06-01' AND fc.Factura_Fecha <= '2024-12-31'
    AND fc.Emp_Id = 1;""")

with engine.connect() as conn:
    df_base = pd.read_sql(query, conn, index_col='Factura_Id')

In [3]:
df_base.head()
print(df_base.count())

Suc_Id                 6543584
Emp_Id                 6543584
Caja_Id                6543584
Factura_Fecha          6543584
Factura_FechaHora      6543584
AnioMes_Id             6543584
Cliente_Id             6543584
MonederoTarj_Id        6543584
Vendedor_Id            6543584
Tipo_Plan              6543584
Edad                   6543584
TipoSucursal_Id        6543584
TipoSucursal_Nombre    6543584
Zona_Id                6543584
Zona_Nombre            6543584
dtype: int64


In [4]:
# Crear una copia del DataFrame original
df = df_base.copy()

# Convertir los valores de hora a cadenas
df['Factura_FechaHora'] = df['Factura_FechaHora'].astype(str)

# Agregar una fecha ficticia a los valores de hora
df['Factura_FechaHora'] = '1900-01-01 ' + df['Factura_FechaHora']

# Convertir la columna Factura_FechaHora a datetime
df['Factura_FechaHora'] = pd.to_datetime(df['Factura_FechaHora'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

# Asegúrate de que Factura_Fecha y Factura_FechaHora sean tipo datetime
df['Factura_Fecha'] = pd.to_datetime(df['Factura_Fecha'])

# Crear el diccionario para mapear los tipos de transacciones
tipos_transaccion = {
    'Monedero tercera edad': 'Tercera Edad',
    'Monedero Cuarta Edad': 'Tercera Edad',
    'Monedero todo publico': 'Todo Publico',
    'Monedero Clínica TP': 'Todo Publico',
    'Monedero Clínica TE': 'Tercera Edad',
    'No Identificada': 'No Identificada',
    'Aseguradoras': 'Aseguradoras'
}

# Mapear los tipos de transacciones y eliminar las no relevantes
df['Tipo_Plan'] = df['Tipo_Plan'].map(tipos_transaccion)
#df = df.dropna(subset=['Tipo_Plan'])

# Extraer hora y día de la semana para el agrupamiento
df['Hora'] = df['Factura_FechaHora'].dt.hour
df['Dia_Semana'] = df['Factura_Fecha'].dt.day_name()

# Agrupar por sucursal, día de la semana, hora y tipo de transacción
agrupado = df.groupby(['Suc_Id', 'Dia_Semana', 'Hora', 'Tipo_Plan','Factura_Fecha', 'TipoSucursal_Nombre']
                      ).size().reset_index(name='Cantidad_Transacciones')

df_filter = df[(df['Suc_Id'] == 1) & (df['Dia_Semana'] == 'Monday') & (df['Hora'] == 7)]

In [5]:
print(df_filter)

            Suc_Id  Emp_Id  Caja_Id Factura_Fecha   Factura_FechaHora  \
Factura_Id                                                              
76765            1       1        2    2024-12-23 1900-01-01 07:28:36   
75297            1       1        2    2024-11-25 1900-01-01 07:35:22   
75296            1       1        2    2024-11-25 1900-01-01 07:25:21   
75658            1       1        2    2024-12-02 1900-01-01 07:58:45   
75657            1       1        2    2024-12-02 1900-01-01 07:51:11   
76045            1       1        2    2024-12-09 1900-01-01 07:58:23   
74573            1       1        2    2024-11-11 1900-01-01 07:40:54   
74574            1       1        2    2024-11-11 1900-01-01 07:55:15   
67376            1       1        2    2024-07-01 1900-01-01 07:38:14   
66210            1       1        2    2024-06-10 1900-01-01 07:26:53   
66211            1       1        2    2024-06-10 1900-01-01 07:49:05   
73883            1       1        2    2024-10-28 1

In [7]:
print(df_tiempos)

           TipoSucursal_Nombre        Tipo_Plan  Tiempo_Promedio
0             Centro Comercial  No Identificada         4.750704
1             Centro Comercial     Tercera Edad         6.573587
2             Centro Comercial     Todo Publico         5.671098
3         Clínica/Hospitalaria     Aseguradoras        19.483333
4         Clínica/Hospitalaria  No Identificada         5.101626
5         Clínica/Hospitalaria     Tercera Edad         7.046594
6         Clínica/Hospitalaria     Todo Publico         6.314681
7   Especialidades Oncologicas  No Identificada         6.311111
8   Especialidades Oncologicas     Tercera Edad         7.185040
9   Especialidades Oncologicas     Todo Publico         0.883333
10        Estación de Servicio     Aseguradoras         7.633333
11        Estación de Servicio  No Identificada         5.758902
12        Estación de Servicio     Tercera Edad         7.183161
13        Estación de Servicio     Todo Publico         6.568534
14                     Ge

In [9]:
# Cargar tiempos por tipo de sucursal y tipo de transacción
df_tiempos = pd.read_csv('promedios_horas_pico_general.csv')

# Crear un diccionario para accesos rápidos a tiempos promedio
tiempos_dict = df_tiempos.set_index(['TipoSucursal_Nombre', 'Tipo_Plan'])['Tiempo_Promedio'].to_dict()

# Pivotar la tabla inicial
pivot_actualizado = agrupado.pivot_table(
    index=['Suc_Id', 'Dia_Semana', 'Hora', 'Factura_Fecha', 'TipoSucursal_Nombre'],
    columns='Tipo_Plan',
    values='Cantidad_Transacciones',
    fill_value=0
).reset_index()

# Calcular tiempo total usando operaciones vectorizadas
for tipo_plan in ['No Identificada', 'Tercera Edad', 'Todo Publico', 'Aseguradoras']:
    pivot_actualizado[tipo_plan + '_Tiempo'] = pivot_actualizado[tipo_plan] * \
        pivot_actualizado['TipoSucursal_Nombre'].map(
            lambda sucursal: tiempos_dict.get((sucursal, tipo_plan), 0)
        )

pivot_actualizado['Tiempo_Total'] = (
    pivot_actualizado[['No Identificada_Tiempo', 'Tercera Edad_Tiempo', 'Todo Publico_Tiempo', 'Aseguradoras_Tiempo']]
    .sum(axis=1)
)

# Calcular personal necesario
pivot_actualizado['Personal_Necesario'] = np.ceil(pivot_actualizado['Tiempo_Total'] / 42)

# Mapear días de la semana
dias_semana_map = {'Monday': 1, 'Tuesday': 2, 'Wednesday': 3, 'Thursday': 4, 'Friday': 5, 'Saturday': 6, 'Sunday': 7}
pivot_actualizado['Dia_Semana'] = pivot_actualizado['Dia_Semana'].map(dias_semana_map)

# Calcular transacciones totales
pivot_actualizado['Transacciones_Totales'] = pivot_actualizado[
    ['Todo Publico', 'Tercera Edad', 'No Identificada', 'Aseguradoras']
].sum(axis=1)

# Seleccionar las columnas finales
pivot_actualizado = pivot_actualizado[['Suc_Id', 'Dia_Semana', 'Hora', 'Transacciones_Totales', 'Personal_Necesario']]

In [10]:
pivot_actualizado.to_csv('personal_necesario.csv', index=False)